# Spark Streaming
This notebook takes in the `gemini-feed` Kafka topic and produces to the `spark.out` topic a feed which includes the order price volume ratio and bid/ask liquidity for BTC.

In [ ]:
!pip install kafka

In [ ]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils  
from kafka import KafkaProducer
import json
import time

In [ ]:
producer = KafkaProducer(bootstrap_servers='kafka-node:9092')

In [ ]:
sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")  
sc.setLogLevel("ERROR") 
ssc = StreamingContext(sc, 60)  

In [ ]:
kafkaStream = KafkaUtils.createStream(ssc, 'kafka-node:2181', 'spark-streaming', {'gemini-feed':1})  

In [ ]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [ ]:
def handler(message):
    records = message.collect()
    for record in records:
        output = {}
        output['type'] = record[0]
        output['value'] = record[1]
        output['timestamp'] = int(time.time())
        producer.send('spark.out', bytes(json.dumps(output), 'utf-8'))
        producer.flush()

In [ ]:
def price_volume(x):
    price = float(x['price'])
    remaining = float(x['remaining'])
    x['price_volume'] = price * remaining
    return (x['side'], x['price_volume'])

Get price/volume ratio

In [ ]:
parsed_pv = parsed.map(lambda x: price_volume(x))

Group by side (bid or ask)

In [ ]:
grouped = parsed_pv.reduceByKey(lambda accum, n: accum + n)

In [ ]:
grouped_sorted = grouped.transform(lambda rdd: rdd.sortByKey())

In [ ]:
grouped_sorted.foreachRDD(handler)

Generate a ratio of the price_volume of ask to bids. So, if ask volume is 4000 and bid volume is 1000 then the ratio is 4. This suggests ask (sell) side liquidity greatly exceeds buy side.

In [ ]:
results = grouped_sorted.map(lambda x: ('price_volume', x[1]))

In [ ]:
ratio = results.reduceByKey(lambda x, y: x/y)

In [ ]:
ratio.foreachRDD(handler)

Print results

In [ ]:
grouped_sorted.pprint()
ratio.pprint()

In [ ]:
ssc.start()

In [ ]:
ssc.stop()